# Figshare API

# Setup

## Instructions

This notebook utilizes the Figshare API. Follow these steps in order to get the necessary credentials to continue:
1. Create a Figshare account at https://figshare.com/account/register
2. After logging in, click on your account photo in the top right corner, and then click on 'Applications'
3. Access API key either by:
    - Create an application by clicking on 'Create Application'
    - Create an API key by clicking on 'Create Personal Token'
4. Load API key:
    - For repeated use, follow the ```pickle_tutorial.ipynb``` instructions to create create a ```./credentials.pkl``` file that holds a dictionary containing the entry ```{'FIGSHARE_TOKEN': MYKEY}```, with MYKEY being your API key.
    - For sparser use, users can run the credentials cell and paste their API key when prompted.

## Additional Information

Documentation Guide:
- Figshare API ([Figshare](https://docs.figshare.com))

## Workflow

Figshare allows a variety of search types. For this script, we search: (1) articles, (2) collections, and (3) projects.

"Articles" in this context include a variety of item types: 1 - Figure, 2 - Media, 3 - Dataset, 5 - Poster, 6 - Journal contribution, 7 - Presentation, 8 - Thesis, 9 - Software, 11 - Online resource, 12 - Preprint, 13 - Book, 14 - Conference contribution, 15 - Chapter, 16 - Peer review, 17 - Educational resource, 18 - Report, 19 - Standard, 20 - Composition, 21 - Funding, 22 - Physical object, 23 - Data management plan, 24 - Workflow, 25 - Monograph, 26 - Performance, 27 - Event, 28 - Service, 29 - Model

Figshare workflow:
1.	Specify search terms in a list
2.	Specify search_types in a list
    - Search can only be conducted over articles, collections, or projects
3.	The function `get_search_output` calls the Figshare API for each combination of search terms (i.e., "machine learning" and search types (i.e., articles, collections, projects)
    - Results are paginated, so for each search term/search type combination, return results from each page of results and combine into a single data frame
    - Results returned are high level only (id, title, doi, URL, etc) – full object metadata is called downstream
4.	Function `get_search_output` is called within `get_search_outputs`
    - This is where the search actually happens
    - This function is separate because (seems like??) it returns results from `get_search_output` with addition of columns indicating search term and search type and search page - but I’m only seeing search_page in the sample_df
5.	Use object-level URL from `get_search_output` results to get full metadata for each object:
    - Function `_retrieve_object_json` calls Figshare API for each URL and returns JSON response
    - Function `get_metadata` actually calls `_retrieve_object_json` to do the search and formats the results as a dataframe 
6.	Not quite sure what the code under “perform metadata extraction” is doing
7.	Running `merge_search_and_metadata_dicts` returns `KeyError: "['search_type'] not in index"`
8.	Everything below “code that loops through search terms and objects” is old and can be deleted


## Imports

In [1]:
import requests # For querying data from API
import pandas as pd # For storing/manipulating query data
from tqdm import tqdm # Gives status bar on loop completion
import itertools # For efficient looping over queries
from collections import OrderedDict

from utils import flatten_nested_df

# For loading credentials
import pickle
import os 

API access tokens have been stored in credentials.pkl file

In [2]:
# Load credentials

# Check for credentials file
try:
    with open('credentials.pkl', 'rb') as credentials:
        FIGSHARE_TOKEN = pickle.load(credentials)['FIGSHARE_TOKEN']
except:
    FIGSHARE_TOKEN = input('Please enter your Figshare API Key: ')

In [3]:
# Search constants
BASE_URL = 'https://api.figshare.com/v2/'
HEADERS = {'Authorization': f'token {FIGSHARE_TOKEN}'}

Figshare allows a variety of search types. For this script, we search: (1) articles, (2) collections, and (3) projects.

"Articles" in this context include a variety of item types: 1 - Figure, 2 - Media, 3 - Dataset, 5 - Poster, 6 - Journal contribution, 7 - Presentation, 8 - Thesis, 9 - Software, 11 - Online resource, 12 - Preprint, 13 - Book, 14 - Conference contribution, 15 - Chapter, 16 - Peer review, 17 - Educational resource, 18 - Report, 19 - Standard, 20 - Composition, 21 - Funding, 22 - Physical object, 23 - Data management plan, 24 - Workflow, 25 - Monograph, 26 - Performance, 27 - Event, 28 - Service, 29 - Model

Overall workflow:
1. Extract object IDs for each combination of search terms (i.e., "machine learning" and search types (i.e., articles, collections, projects)

# Data Wrangling

## Extracting Object ID's

In [4]:
def get_search_output(search_term, search_type, flatten=False):
    """
    Calls the Figshare API with the specified search term and returns the search output results.
    
    Params:
    - search_term (str): keyword to seach for
    - search_type (str): objects to search over (must be either datasets or kernels)
    - flatten (bool): optional (default=False)
   
    Returns:
    - df (pandas.DataFrame): DataFrame containing the output of the search query
    """
    
    # Make sure our input is valid
    assert isinstance(search_term, str), 'Search term must be a string'
    assert search_type in ('articles', 'collections', 'projects'), \
        'Search can only be conducted over articles, collections, or projects'
        
    # Set search variables
    start_page = 1
    page_size = 1000 # Maximum page size (min = 10)
    output = None
    search_df = pd.DataFrame()
    
    search_params = {
        'search_for': search_term,
        'page': start_page, 
        'page_size': page_size,  
        }
        
    search_url = f'{BASE_URL}/{search_type}'

    ## Run search for public articles
    response = requests.get(search_url, params=search_params, headers=HEADERS)

    ## Put output into json format
    output = response.json()
    
    # Continue searching until we reach an empty page
    while output != []:
        # Turn outputs into DataFrame & add page info
        output_df = pd.DataFrame(output)
        output_df['search_page'] = search_params['page']
        
        # Append modified output df to our cumulative search DataFrame
        search_df = pd.concat([search_df, output_df])
        
        # Increment page number to query
        search_params['page'] += 1

        ## Run search for public articles
        response = requests.get(search_url, params=search_params, headers=HEADERS)

        ## Put output into json format
        output = response.json()
    
    if flatten:
        search_df = flatten_nested_df(search_df)
    
    return search_df

In [5]:
#needs name that is more different from above function

def get_search_outputs(search_terms, search_types, flatten=False):
    """
    Call the Figshare API for each search term and search type. 
    Results are retured in results['{term}_{type}'] = df
    
    Params:
    - search_terms (list-like): collection of search terms to query over
    - search_types (list-like): collection of search types to query over
    - flatten (bool): optional, (default=False)
    
    Returns:
    - results (dict): dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    num_searches = len(search_terms) * len(search_types)
    results = OrderedDict()

    for search_term, search_type in itertools.product(search_terms, search_types):
        results[(search_term, search_type)] = get_search_output(search_term, search_type, flatten)
        
    return results

### Perform Search

In [6]:
search_terms = ['iguana']
search_types = ['collections', 'projects', 'articles']

In [7]:
search_output_dict = get_search_outputs(search_terms, search_types, flatten=True)

In [8]:
sample_key = (search_terms[0], search_types[0])
sample_df = search_output_dict[sample_key]

In [9]:
sample_df.head()

,id,title,doi,handle,url,published_date,timeline_posted,search_page
0,3582815,Data from: Vascular patterns in iguanas and ot...,10.5061/dryad.27m63.2,,https://api.figshare.com/v2/collections/3582815,2016-11-25T19:47:11Z,2016-11-25T19:47:11,1
1,4755440,Data from: Vascular patterns in iguanas and ot...,10.5061/dryad.27m63,,https://api.figshare.com/v2/collections/4755440,2019-11-26T08:07:43Z,2019-11-26T08:07:43,1
2,4596320,Data from: Vascular patterns in iguanas and ot...,10.5061/dryad.27m63.1,,https://api.figshare.com/v2/collections/4596320,2019-07-30T16:44:02Z,2019-07-30T16:44:02,1
3,5234804,First known trace fossil of a nesting iguana (...,10.1371/journal.pone.0242935,,https://api.figshare.com/v2/collections/5234804,2020-12-09T18:32:22Z,2020-12-09T18:32:22,1
4,5311858,Systemic <i>Helicobacter</i> infection and ass...,10.1371/journal.pone.0247010,,https://api.figshare.com/v2/collections/5311858,2021-02-19T18:33:04Z,2021-02-19T18:33:04,1


## Get Metadata

In [10]:
def _retrieve_object_json(object_url):
    '''
    Queries Figshare for object data (json file) & returns the json data as a dictionary
    
    Params:
    - object_url (str): path for the dataset
    
    Returns:
    - object_data_dict (dict): dictionary containing json data
    '''
    
    # Download the metadata
    response = requests.get(object_url, headers=HEADERS)
    json_data = response.json()
    
    return json_data

In [11]:
def get_metadata(object_paths, flatten=False):
    """
    Retrieves the metadata for the object/objects listed in object_paths
    
    Params:
    - object_paths (str/list-like): string or list of strings containing the paths for the objects
    - flatten (bool): optional, (default=False)
    
    Returns:
    - metadata_df (pandas.DataFrame): DataFrame containing metadata for the requested objects
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
    
    #create empty pandas dataframe to put results in
    metadata_df = pd.DataFrame()

    #for each path, get full object details
    for object_path in tqdm(object_paths):
        #URL syntax for object details is: https://api.figshare.com/v2/{search_type}/{object_id}        
        json_data = _retrieve_object_json(object_path)
        
        #appending json collapses first level, which is a start
        #for now, can leave files, custom fields, author, etc as list of dictionary
        metadata_df = metadata_df.append(json_data, ignore_index=True)

    if flatten:
        metadata_df = flatten_nested_df(metadata_df)
        
    return metadata_df

### Perform Metadata Extraction

In [12]:
## Extract IDs from DataFrame, and returns as list of strings
metadata_dict = OrderedDict()

for query, df in search_output_dict.items():
    # Create object paths
    _, search_type = query
    object_ids = df.id.convert_dtypes(convert_string=True).tolist()
    object_paths = [f'{BASE_URL}/{search_type}/{object_id}' for object_id in object_ids]
    
    metadata_dict[query] = get_metadata(object_paths, flatten=True)

100%|██████████| 152/152 [02:12<00:00,  1.15it/s]


## Combining Results

In [13]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on, save=False):
    """
    Merges together all of the search and metadata DataFrames by the given 'on' key
    
    Params:
    - search_dict (dict): dictionary of search output results
    - metadata_dict (dict): dictionary of metadata results
    - on (str/list-like): column name(s) to merge the two dicts on
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/figshare/figshare_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain enough strings (one per query; len(search_terms) * len(search_types))
            
    Returns:
    - df_dict (OrderedDict): OrderedDict containing all of the merged search/metadata dicts
    """

    # Make sure the dictionaries contain the same searches
    assert search_dict.keys() == metadata_dict.keys(), 'Dictionaries must contain the same searches'
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')

    # Ensure the on variable is proper
    try:
        assert len(on) == 2 or isinstance(on, str)
        if (len(on) == 2) and (not isinstance(on, str)):
            left_on, right_on = on
            on = None
    except:
        raise ValueError('Incorrect value of "on" passed')
        
    # Merge the DataFrames
    df_dict = OrderedDict()
    for (query_key, search_df), (query_key, metadata_df), save_loc in zip(search_dict.items(), 
                                                                          metadata_dict.items(), 
                                                                          save):
        # Keep just search info, id and timeline from initial extract 
        # Timeline only present in some search types
        columns_to_keep = ['id', 'search_page']
        
        if 'timeline' in search_df.columns:
            columns_to_keep.append('timeline')
            
        search_df = search_df[columns_to_keep]

        #Merge small version of "full" dataframe with "detailed" dataframe
        if on: # only one value to merge on
            df_all = pd.merge(search_df, metadata_df, on=on, how='inner')
        else:
            df_all = pd.merge(search_df, metadata_df, left_on=left_on, right_on=right_on)
            
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'figshare')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure figshare directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError(f'Save type must be bool or str, not {type(save_loc)}')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

In [14]:
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict, on='id')

## Code that loops through search terms and objects

### also an issue with projects objects that stops code

In [15]:
search_type = 'collections'
search_term = 'iguana'
search_url = f'https://api.figshare.com/v2/{search_type}'

PARAMS = {
        'search_for': search_term, #search term
        'page': 1, 
        'page_size': 10,  
        }

response = requests.get(search_url, params=PARAMS, headers=HEADERS)

In [16]:
#FIGSHARE_TOKEN imported from credentials.pkl


#List of search terms
SEARCH_TERMS = ['iguana']

#List of Figshare object types to search
SEARCH_TYPES = ['articles', 'collections', 'projects']

# Set dummy json output for page loop
output = None

###### LOOP 1 - loop through search terms ######
for search_term in SEARCH_TERMS:
    print('Searching:', search_term)

    #Specify page to return with search (results are paginated) - TO DO: FIX HARD CODING
    PAGE = 1
    
    #Specify number of results included on a page (default is 10, max is 1000) - TO DO: FIX HARD CODING
    PAGE_SIZE = 1000

    #Set params term
    PARAMS = {
        'search_for': search_term, #search term
        'page': PAGE, 
        'page_size': PAGE_SIZE,  
        }
    
    ###### LOOP 2 - loop through object types ######

    for search_type in SEARCH_TYPES:
        print(f'\tSearching over: {search_type}')
        
        URL_j = f'https://api.figshare.com/v2/{search_type}'

        ## Run search for public articles
        response = requests.get(URL_j, params=PARAMS, headers=HEADERS)

        ## Put output into json format
        output = response.json()
        
        # Continue searching until we reach an empty page
        while output != []:
            #Convert output to pd dataframe and see table format
            df_full = pd.DataFrame(output)

            ## Extract IDs
            full_ids = list(df_full.id)

            ####### LOOP 3 - loop to to extract object  details by object ID ######

            #create empty pandas dataframe to put results in
            df_detailed = pd.DataFrame()

            #for each ID, get full object details
            for i in full_ids:
                id_i = str(i)
                #URL syntax for object details is: https://api.figshare.com/v2/{search_type}/{object_id}
                URL_i = f'{URL_j}/{id_i}'
                response_i = requests.get(URL_i, headers=HEADERS)
                output_i = response_i.json()
                #appending json collapses first level, which is a start
                #for now, can leave files, custom fields, author, etc as list of dictionary
                df_detailed = df_detailed.append(output_i, ignore_index=True)

                #Keep just id and timeline from initial extract (all others are in detailed extract)
                #'timeline' is NOT in projects, so...do we really need it?
                df_small = df_full[['id','timeline']]

                #Add info about which search type and page this is associated with
                df_detailed['Search_type'] = search_type
                df_detailed['search_page'] = PARAMS['page']

                #Merge small version of "full" dataframe with "detailed" dataframe
                df_all = pd.merge(df_small, df_detailed, on='id', how='inner')

            #Write to csv
            output_file = f'Figshare_{search_term}_{search_type}.csv'
            df_all.to_csv(output_file, index=False)
            
            # Increment page number to query
            PARAMS['page'] += 1
            
            ## Run search for public articles
            response = requests.get(URL_j, params=PARAMS, headers=HEADERS)
            
            ## Put output into json format
            output = response.json()

    print(f'Finished {search_term} search')

Searching: iguana
	Searching over: articles
	Searching over: collections
	Searching over: projects
Finished iguana search


In [17]:
df_all.head()

,id,timeline_x,authors,categories,citation,confidential_reason,created_date,custom_fields,defined_type,defined_type_name,...,timeline_y,title,url,url_private_api,url_private_html,url_public_api,url_public_html,version,Search_type,search_page
0,9172331,"{'posted': '2019-07-30T16:44:01', 'publisherPu...","[{'id': 813288, 'full_name': 'William Ruger Po...","[{'id': 146, 'title': 'Space Science', 'parent...","Ruger Porter, William; M. Witmer, Lawrence (20...",,2019-07-30T16:44:01Z,"[{'name': 'dc.type.embargo', 'value': ''}, {'n...",3.0,dataset,...,"{'posted': '2019-07-30T16:44:01', 'publisherPu...","CT scan data of green iguana, Iguana iguana OU...",https://api.figshare.com/v2/articles/9172331,https://api.figshare.com/v2/account/articles/9...,https://figshare.com/account/articles/9172331,https://api.figshare.com/v2/articles/9172331,https://dryad.figshare.com/articles/dataset/CT...,1.0,articles,1
1,9172307,"{'posted': '2019-07-30T16:42:42', 'publisherPu...","[{'id': 813288, 'full_name': 'William Ruger Po...","[{'id': 146, 'title': 'Space Science', 'parent...","Ruger Porter, William; M. Witmer, Lawrence (20...",,2019-07-30T16:42:42Z,"[{'name': 'dc.type.embargo', 'value': ''}, {'n...",3.0,dataset,...,"{'posted': '2019-07-30T16:42:42', 'publisherPu...","CT scan data of green iguana, Iguana iguana OU...",https://api.figshare.com/v2/articles/9172307,https://api.figshare.com/v2/account/articles/9...,https://figshare.com/account/articles/9172307,https://api.figshare.com/v2/articles/9172307,https://dryad.figshare.com/articles/dataset/CT...,1.0,articles,1
2,9172310,"{'posted': '2019-07-30T16:42:56', 'publisherPu...","[{'id': 813288, 'full_name': 'William Ruger Po...","[{'id': 146, 'title': 'Space Science', 'parent...","Ruger Porter, William; M. Witmer, Lawrence (20...",,2019-07-30T16:42:56Z,"[{'name': 'dc.type.embargo', 'value': ''}, {'n...",3.0,dataset,...,"{'posted': '2019-07-30T16:42:56', 'publisherPu...","CT scan data of green iguana, Iguana iguana OU...",https://api.figshare.com/v2/articles/9172310,https://api.figshare.com/v2/account/articles/9...,https://figshare.com/account/articles/9172310,https://api.figshare.com/v2/articles/9172310,https://dryad.figshare.com/articles/dataset/CT...,1.0,articles,1
3,9172322,"{'posted': '2019-07-30T16:43:36', 'publisherPu...","[{'id': 813288, 'full_name': 'William Ruger Po...","[{'id': 4, 'title': 'Biochemistry', 'parent_id...","Ruger Porter, William; M. Witmer, Lawrence (20...",,2019-07-30T16:43:36Z,"[{'name': 'dc.type.embargo', 'value': ''}, {'n...",3.0,dataset,...,"{'posted': '2019-07-30T16:43:36', 'publisherPu...","CT scan data of green iguana, Iguana iguana OU...",https://api.figshare.com/v2/articles/9172322,https://api.figshare.com/v2/account/articles/9...,https://figshare.com/account/articles/9172322,https://api.figshare.com/v2/articles/9172322,https://dryad.figshare.com/articles/dataset/CT...,1.0,articles,1
4,9172319,"{'posted': '2019-07-30T16:43:12', 'publisherPu...","[{'id': 813288, 'full_name': 'William Ruger Po...","[{'id': 146, 'title': 'Space Science', 'parent...","Ruger Porter, William; M. Witmer, Lawrence (20...",,2019-07-30T16:43:12Z,"[{'name': 'dc.type.embargo', 'value': ''}, {'n...",3.0,dataset,...,"{'posted': '2019-07-30T16:43:12', 'publisherPu...","CT scan data of green iguana, Iguana iguana OU...",https://api.figshare.com/v2/articles/9172319,https://api.figshare.com/v2/account/articles/9...,https://figshare.com/account/articles/9172319,https://api.figshare.com/v2/articles/9172319,https://dryad.figshare.com/articles/dataset/CT...,1.0,articles,1


## OLD CODE
#### Specify search parameters

In [18]:
#FIGSHARE_TOKEN imported from credentials.pkl
HEADERS = {'Authorization': 'token '+ FIGSHARE_TOKEN}

#Specify search terms
SEARCH = 'machine learning'

#turn into list for loop: SEARCH = ['machine learning'] 
#with whatever search terms we decide to use

#Specify which search (collections, articles, projects, data)
ARTICLE_URL = 'https://api.figshare.com/v2/articles'
COLLECTIONS_URL = "https://api.figshare.com/v2/collections"
PROJECTS_URL = "https://api.figshare.com/v2/projects"

#turn into list for loop: SEARCH_TYPES = ['articles', 'collections', 'projects']
#for j in SEARCH_TYPES:
    #URL_j = "https://api.figshare.com/v2/" + SEARCH_TYPES_j
    
#article search should include datasets
#Only return articles with the respective type. Mapping for item_type is: 
#1 - Figure, 2 - Media, 3 - Dataset, 5 - Poster, 6 - Journal contribution, 7 - Presentation, 
#8 - Thesis, 9 - Software, 11 - Online resource, 12 - Preprint, 13 - Book, 14 - Conference contribution, 
#15 - Chapter, 16 - Peer review, 17 - Educational resource, 18 - Report, 19 - Standard, 20 - Composition, 
#21 - Funding, 22 - Physical object, 23 - Data management plan, 24 - Workflow, 
#25 - Monograph, 26 - Performance, 27 - Event, 28 - Service, 29 - Model

#should make a df with this and merge to get more informative item_type column in final output

#Specify page to return with search (results are paginated)
PAGE = 1

#Specify number of results included on a page (default is 10, max is 1000)
PAGE_SIZE = 10

## Could iterate through pages until get response: {'message': 'Bad Request', 'code': 'BadRequest'} 
## Seems like there should be a better way

#Specify page and page size parameters
#Other search options are available, including limit and offset, but at the moment page/page size seem most useful
    #if set both page/page size and limit/offset, get:
    #{'message': 'Pagination options can be set either via page/page_size or limit/offset params','code': 'ConflictingPaginationOptions'}
    
    #'limit': 1000, #Number of results included on a page. Used for pagination with query (optional) - not sure how differs from page_size
    #'offset': 1 #, #Where to start the listing(the offset of the first result). Used for pagination with limit (optional)
#We don't need to specify other search parameters like institution, group, modified since, etc.

#Full search parameters
PARAMS = {
    'search_for': SEARCH, #search term
    'page': PAGE, 
    'page_size': PAGE_SIZE,  
    }

## Draft workflow: search public articles

In [19]:
## Run search for public articles
response = requests.get(ARTICLE_URL, params=PARAMS, headers=HEADERS)

## Put output into json format
output = response.json()

#Convert output to pd dataframe and see table format
df_full = pd.DataFrame(output)

In [20]:
## See what output looks like
output

[{'id': 6716609,
  'title': 'Expressive Collaborative Music Performance via Machine Learning',
  'doi': '10.1184/r1/6716609.v1',
  'handle': '',
  'url': 'https://api.figshare.com/v2/articles/6716609',
  'published_date': '2016-08-01T00:00:00Z',
  'thumb': 'https://s3-eu-west-1.amazonaws.com/ppreviews-cmu-49810598590254/12249995/thumb.png',
  'defined_type': 8,
  'defined_type_name': 'thesis',
  'group_id': 18614,
  'url_private_api': 'https://api.figshare.com/v2/account/articles/6716609',
  'url_public_api': 'https://api.figshare.com/v2/articles/6716609',
  'url_private_html': 'https://figshare.com/account/articles/6716609',
  'url_public_html': 'https://kilthub.cmu.edu/articles/thesis/Expressive_Collaborative_Music_Performance_via_Machine_Learning/6716609',
  'timeline': {'posted': '2016-08-01T00:00:00',
   'firstOnline': '2018-07-01T00:04:06',
   'revision': '2018-07-01T00:04:07'},
  'resource_title': '',
  'resource_doi': ''},
 {'id': 6714890,
  'title': 'Attribute Learning using J

In [21]:
## See what df looks like
df_full

,id,title,doi,handle,url,published_date,thumb,defined_type,defined_type_name,group_id,url_private_api,url_public_api,url_private_html,url_public_html,timeline,resource_title,resource_doi
0,6716609,Expressive Collaborative Music Performance via...,10.1184/r1/6716609.v1,,https://api.figshare.com/v2/articles/6716609,2016-08-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6716609,https://figshare.com/account/articles/6716609,https://kilthub.cmu.edu/articles/thesis/Expres...,"{'posted': '2016-08-01T00:00:00', 'firstOnline...",,
1,6714890,Attribute Learning using Joint Human and Machi...,10.1184/r1/6714890.v1,,https://api.figshare.com/v2/articles/6714890,2012-08-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6714890,https://figshare.com/account/articles/6714890,https://kilthub.cmu.edu/articles/thesis/Attrib...,"{'posted': '2012-08-01T00:00:00', 'firstOnline...",,
2,6720413,"Learning with Sparcity: Structures, Optimizati...",10.1184/r1/6720413.v1,,https://api.figshare.com/v2/articles/6720413,2013-07-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720413,https://figshare.com/account/articles/6720413,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2013-07-01T00:00:00', 'firstOnline...",,
3,6720908,On Learning from Collective Data,10.1184/r1/6720908.v1,,https://api.figshare.com/v2/articles/6720908,2013-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720908,https://figshare.com/account/articles/6720908,https://kilthub.cmu.edu/articles/thesis/On_Lea...,"{'posted': '2013-12-01T00:00:00', 'firstOnline...",,
4,6720416,Learning with Staleness,10.1184/r1/6720416.v1,,https://api.figshare.com/v2/articles/6720416,2018-03-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720416,https://figshare.com/account/articles/6720416,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2018-03-01T00:00:00', 'firstOnline...",,
5,6720878,Nonparametric Learning in High Dimensions,10.1184/r1/6720878.v1,,https://api.figshare.com/v2/articles/6720878,2010-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720878,https://figshare.com/account/articles/6720878,https://kilthub.cmu.edu/articles/thesis/Nonpar...,"{'posted': '2010-12-01T00:00:00', 'firstOnline...",,
6,7553882,Distribution and Histogram (DisH) Learning,10.1184/r1/7553882.v1,,https://api.figshare.com/v2/articles/7553882,2018-07-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/7...,https://api.figshare.com/v2/articles/7553882,https://figshare.com/account/articles/7553882,https://kilthub.cmu.edu/articles/thesis/Distri...,"{'posted': '2018-07-01T00:00:00', 'revision': ...",,
7,6720509,Mathematical Theories of Interaction with Oracles,10.1184/r1/6720509.v1,,https://api.figshare.com/v2/articles/6720509,2013-10-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720509,https://figshare.com/account/articles/6720509,https://kilthub.cmu.edu/articles/thesis/Mathem...,"{'posted': '2013-10-01T00:00:00', 'firstOnline...",,
8,6720836,New Paradigms and Optimality Guarantees in Sta...,10.1184/r1/6720836.v1,,https://api.figshare.com/v2/articles/6720836,2017-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c..

In [22]:
## Extract IDs
full_ids = list(df_full.id)

In [23]:
## Loop to extract article details by object ID

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}

#create empty pandas dataframe
df_detailed = pd.DataFrame()

#for each ID, 
for i in full_ids:
    id_i = str(i)
    URL_i = 'https://api.figshare.com/v2/articles/' + id_i
    #print(URL_i)
    response_i = requests.get(URL_i, headers=HEADERS)
    json_i = response_i.json()
    #json_normalize collapses first level, which is a start
    #for now, can leave files, custom fields, author, etc as list of dictionary
    df_i = pd.json_normalize(json_i)
    df_detailed = df_detailed.append(df_i)

In [24]:
df_detailed

,files,custom_fields,authors,figshare_url,description,funding,funding_list,version,status,size,...,url_public_html,resource_title,resource_doi,license.value,license.name,license.url,timeline.posted,timeline.firstOnline,timeline.revision,timeline.submission
0,"[{'id': 12249995, 'name': 'Expressive Collabor...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5464304, 'full_name': 'Guangyu Xia', ...",https://kilthub.cmu.edu/articles/thesis/Expres...,<p>Techniques of Artificial Intelligence and H...,None,[],1,public,11070988,...,https://kilthub.cmu.edu/articles/thesis/Expres...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2016-08-01T00:00:00,2018-07-01T00:04:06,2018-07-01T00:04:07,NaN
0,"[{'id': 12247874, 'name': 'Attribute Learning ...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5364275, 'full_name': 'Edith L.M. Law'...",https://kilthub.cmu.edu/articles/thesis/Attrib...,<p>This thesis is centered around the problem ...,None,[],1,public,8863551,...,https://kilthub.cmu.edu/articles/thesis/Attrib...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-08-01T00:00:00,2018-06-30T23:16:04,2018-06-30T23:16:05,NaN
0,"[{'id': 12254777, 'name': 'Learning with Sparc...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362001, 'full_name': 'Xi Chen', 'is_...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>The development of modern information techn...,None,[],1,public,3773851,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-07-01T00:00:00,2018-07-01T00:25:42,2018-07-01T00:25:43,NaN
0,"[{'id': 12255338, 'name': 'On Learning from Co...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5363849, 'full_name': 'Liang Xiong', ...",https://kilthub.cmu.edu/articles/thesis/On_Lea...,In many machine learning problems and applicat...,None,[],1,public,4152506,...,https://kilthub.cmu.edu/articles/thesis/On_Lea...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-12-01T00:00:00,2018-07-01T00:48:03,2018-07-01T00:48:04,NaN
0,"[{'id': 12254780, 'name': 'Learning with Stale...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4947985, 'full_name': 'Wei Dai', 'is_a...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>A fundamental assumption behind most machin...,None,[],1,public,4664211,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-03-01T00:00:00,2018-07-01T00:25:52,2018-07-01T00:25:53,NaN
0,"[{'id': 12255305, 'name': 'Nonparametric Learn...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362709, 'full_name': 'Han Liu', 'is_...",https://kilthub.cmu.edu/articles/thesis/Nonpar...,This thesis develops flexible and principled n...,None,[],1,public,4835357,...,https://kilthub.cmu.edu/articles/thesis/Nonpar...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2010-12-01T00:00:00,2018-07-01T00:46:48,2018-07-01T00:46:50,NaN
0,"[{'id': 14038886, 'name': 'joliva_MachineLearn...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4944283, 'full_name': 'Junier Oliva', ...",https://kilthub.cmu.edu/articles/thesis/Distri...,Machine learning has made incredible advances ...,None,[],1,public,18600219,...,https://kilthub.cmu.edu/articles/thesis/Distri...,,,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-07-01T00:00:00,2019-01-16T19:30:34,2019-01-16T19:30:35,2019-01-07T15:04:02
0,"[{'id': 12254909, 'name': 'Mathematical Theori...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5361983, 'full_name': 'Liu Yang', 'is...",https://kilthub.cmu.edu/articles/thesis/Mathem...,Machine Learning,None,[],1,public,2102816,...,https://kilthub.cmu.edu/articles/thesis/Mathem...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-10-01T00:00:00,2018-07-01T00:30:15,2018-07-01T00:30:16,NaN
0,"[{'id': 

### Compare high level metadata extract with detailed extract

In [25]:
## Column names in the "full" initial API call
df_full.columns

Index(['id', 'title', 'doi', 'handle', 'url', 'published_date', 'thumb',
       'defined_type', 'defined_type_name', 'group_id', 'url_private_api',
       'url_public_api', 'url_private_html', 'url_public_html', 'timeline',
       'resource_title', 'resource_doi'],
      dtype='object')

In [26]:
## Column names in the "detailed" API call
df_detailed.columns

Index(['files', 'custom_fields', 'authors', 'figshare_url', 'description',
       'funding', 'funding_list', 'version', 'status', 'size', 'created_date',
       'modified_date', 'is_public', 'is_confidential', 'is_metadata_record',
       'confidential_reason', 'metadata_reason', 'tags', 'categories',
       'references', 'has_linked_file', 'citation', 'is_embargoed',
       'embargo_date', 'embargo_type', 'embargo_title', 'embargo_reason',
       'embargo_options', 'id', 'title', 'doi', 'handle', 'url',
       'published_date', 'thumb', 'defined_type', 'defined_type_name',
       'group_id', 'url_private_api', 'url_public_api', 'url_private_html',
       'url_public_html', 'resource_title', 'resource_doi', 'license.value',
       'license.name', 'license.url', 'timeline.posted',
       'timeline.firstOnline', 'timeline.revision', 'timeline.submission'],
      dtype='object')

In [27]:
## What do these have in common?
overlap = list(set(df_full).intersection(set(df_detailed)))
overlap

['doi',
 'published_date',
 'url_private_api',
 'resource_title',
 'defined_type_name',
 'thumb',
 'url_public_api',
 'defined_type',
 'url_private_html',
 'group_id',
 'id',
 'url_public_html',
 'url',
 'handle',
 'resource_doi',
 'title']

In [28]:
## Are they the same?
sorted(df_full.columns) == sorted(overlap)

False

In [29]:
## So which ones are in one but not the other?
sorted(df_full.columns) 

['defined_type',
 'defined_type_name',
 'doi',
 'group_id',
 'handle',
 'id',
 'published_date',
 'resource_doi',
 'resource_title',
 'thumb',
 'timeline',
 'title',
 'url',
 'url_private_api',
 'url_private_html',
 'url_public_api',
 'url_public_html']

In [30]:
sorted(overlap)

['defined_type',
 'defined_type_name',
 'doi',
 'group_id',
 'handle',
 'id',
 'published_date',
 'resource_doi',
 'resource_title',
 'thumb',
 'title',
 'url',
 'url_private_api',
 'url_private_html',
 'url_public_api',
 'url_public_html']

In [31]:
# timeline is not in detailed extract, but it's the only one that's not the same

### Combine full and detailed API extracts by object ID

In [32]:
## Keep just id and timeline from initial extract (all others are in detailed extract)
## CHECK IF TIMELINE VAR IS IN COLLECTIONS AND PROJECT OBJECTS
df_small = df_full[['id','timeline']]

df_small

,id,timeline
0,6716609,"{'posted': '2016-08-01T00:00:00', 'firstOnline..."
1,6714890,"{'posted': '2012-08-01T00:00:00', 'firstOnline..."
2,6720413,"{'posted': '2013-07-01T00:00:00', 'firstOnline..."
3,6720908,"{'posted': '2013-12-01T00:00:00', 'firstOnline..."
4,6720416,"{'posted': '2018-03-01T00:00:00', 'firstOnline..."
5,6720878,"{'posted': '2010-12-01T00:00:00', 'firstOnline..."
6,7553882,"{'posted': '2018-07-01T00:00:00', 'revision': ..."
7,6720509,"{'posted': '2013-10-01T00:00:00', 'firstOnline..."
8,6720836,"{'posted': '2017-12-01T00:00:00', 'firstOnline..."
9,8336783,"{'posted': '2019-06-27T15:57:04', 'revision': ..."


In [33]:
## Merge small version of "full" dataframe with "detailed" dataframe
df_all = pd.merge(df_small, df_detailed, on = "id", how = "inner")

df_all

,id,timeline,files,custom_fields,authors,figshare_url,description,funding,funding_list,version,...,url_public_html,resource_title,resource_doi,license.value,license.name,license.url,timeline.posted,timeline.firstOnline,timeline.revision,timeline.submission
0,6716609,"{'posted': '2016-08-01T00:00:00', 'firstOnline...","[{'id': 12249995, 'name': 'Expressive Collabor...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5464304, 'full_name': 'Guangyu Xia', ...",https://kilthub.cmu.edu/articles/thesis/Expres...,<p>Techniques of Artificial Intelligence and H...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Expres...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2016-08-01T00:00:00,2018-07-01T00:04:06,2018-07-01T00:04:07,NaN
1,6714890,"{'posted': '2012-08-01T00:00:00', 'firstOnline...","[{'id': 12247874, 'name': 'Attribute Learning ...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5364275, 'full_name': 'Edith L.M. Law'...",https://kilthub.cmu.edu/articles/thesis/Attrib...,<p>This thesis is centered around the problem ...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Attrib...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-08-01T00:00:00,2018-06-30T23:16:04,2018-06-30T23:16:05,NaN
2,6720413,"{'posted': '2013-07-01T00:00:00', 'firstOnline...","[{'id': 12254777, 'name': 'Learning with Sparc...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362001, 'full_name': 'Xi Chen', 'is_...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>The development of modern information techn...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-07-01T00:00:00,2018-07-01T00:25:42,2018-07-01T00:25:43,NaN
3,6720908,"{'posted': '2013-12-01T00:00:00', 'firstOnline...","[{'id': 12255338, 'name': 'On Learning from Co...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5363849, 'full_name': 'Liang Xiong', ...",https://kilthub.cmu.edu/articles/thesis/On_Lea...,In many machine learning problems and applicat...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/On_Lea...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-12-01T00:00:00,2018-07-01T00:48:03,2018-07-01T00:48:04,NaN
4,6720416,"{'posted': '2018-03-01T00:00:00', 'firstOnline...","[{'id': 12254780, 'name': 'Learning with Stale...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4947985, 'full_name': 'Wei Dai', 'is_a...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>A fundamental assumption behind most machin...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-03-01T00:00:00,2018-07-01T00:25:52,2018-07-01T00:25:53,NaN
5,6720878,"{'posted': '2010-12-01T00:00:00', 'firstOnline...","[{'id': 12255305, 'name': 'Nonparametric Learn...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362709, 'full_name': 'Han Liu', 'is_...",https://kilthub.cmu.edu/articles/thesis/Nonpar...,This thesis develops flexible and principled n...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Nonpar...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2010-12-01T00:00:00,2018-07-01T00:46:48,2018-07-01T00:46:50,NaN
6,7553882,"{'posted': '2018-07-01T00:00:00', 'revision': ...","[{'id': 14038886, 'name': 'joliva_MachineLearn...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4944283, 'full_name': 'Junier Oliva', ...",https://kilthub.cmu.edu/articles/thesis/Distri...,Machine learning has made incredible advances ...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Distri...,,,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-07-01T00:00:00,2019-01-16T19:30:34,2019-01-16T19:30:35,2019-01-07T15:04:02
7,6720509,"{'posted': '2013-10-01T00:00:00', 'firstOnline...","[{'id': 12254909, 'name': 'Mathematical Theori...","[{'name': 'Publishe

In [34]:
## List files associated with public articles (by ID)

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}/files

#looks like files are already pulled from article details, so don't need a separate call - all info should already be there
#at some point go back and confirm that there's no new info in files API call